In [3]:
import os
from aesim.simba import ProjectRepository, ThermalData
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from ipywidgets import widgets
import numpy as np

In [4]:
Kp = widgets.FloatSlider(
    value=1.5,  # Initial value
    min=0.1,    # Minimum value
    max=10,    # Maximum value
    step=0.2,   # Step size
    description='Kp:',  # Label for the slider
    orientation='horizontal'  # Orientation of the slider
)
Ki = widgets.FloatSlider(
    value=15000,  # Initial value
    min=5000,    # Minimum value
    max=25000,    # Maximum value
    step=500,   # Step size
    description='Ki:',  # Label for the slider
    orientation='horizontal'  # Orientation of the slider
)

output_widget = widgets.Output()
box1 = widgets.VBox([Kp, Ki])
display(box1)

def handle_slider_change(change):
    with output_widget:
        clear_output(wait=True)
        kp = Kp.value
        ki = Ki.value
        print('kp={:.2f}'.format(kp))
        print('ki={:.0f}'.format(ki))
        file_path = os.path.join(os.getcwd(), "SliderWidget_BuckAverageCM.jsimba")
        project = ProjectRepository(file_path)
        design = project.GetDesignByName('DC-DC Buck Average Current-Mode IGBT')
        design.TransientAnalysis.StopAtSteadyState = True
        PI = design.Circuit.GetDeviceByName('PID1')
        PI.Kp = kp
        PI.Ki = ki
        
        job = design.TransientAnalysis.NewJob()
        status = job.Run()

        V_out = job.GetSignalByName('R1 - Voltage').DataPoints
        I_out = job.GetSignalByName('R1 - Current').DataPoints
        time= job.TimePoints
        
        fig, ax = plt.subplots()
        ax.set_title("Output - Voltage (V)")
        ax.set_ylabel('Voltage (V)')
        ax.set_xlabel('time (s)')
        ax.plot(time,V_out)
        plt.grid(True)
        plt.show()

Kp.observe(handle_slider_change, names='value')
Ki.observe(handle_slider_change, names='value')
display(output_widget)

Output()